In [1]:
import pandas as pd
from scipy.optimize import curve_fit
import seaborn
seaborn.set(style='whitegrid')
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import LogFormatter
%matplotlib qt
# %matplotlib inline
from datetime import datetime, date

C:\Users\etiirine\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Reading in the data

In [2]:
#read in the data
mid=pd.read_csv("MID_series2.tsv", sep="\t", skiprows=lambda x: x <=11,  on_bad_lines="skip")
#add elapsed time columns
timestamps=pd.to_datetime(mid["mass 2_time"],format="%Y/%m/%d %H:%M:%S")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
mid.insert(1,"Elapsed time, mass 2",runtime)

mid.head()


,mass 2_time,"Elapsed time, mass 2",mass 2_value,mass 4_time,mass 4_value,mass 12_time,mass 12_value,mass 14_time,mass 14_value,mass 16_time,mass 16_value,mass 18_time,mass 18_value,mass 28_time,mass 28_value,mass 40_time,mass 40_value,mass 44_time,mass 44_value,Unnamed: 18
0,2022/11/08 10:57:36,0.0,2.150000e-13,2022/11/08 10:57:37,2.150000e-13,2022/11/08 10:57:42,1.970000e-14,2022/11/08 10:57:43,1.960000e-14,2022/11/08 10:57:44,1.096000e-13,2022/11/08 10:57:45,8.600000e-15,2022/11/08 10:57:51,1.930000e-14,NaN,NaN,NaN,NaN,NaN
1,2022/11/08 10:57:55,19.0,1.935000e-13,2022/11/08 10:58:00,6.390000e-14,2022/11/08 10:58:03,2.660000e-14,2022/11/08 10:58:04,1.391000e-13,2022/11/08 10:58:05,7.500000e-14,2022/11/08 10:58:06,5.590000e-13,2022/11/08 10:58:13,9.675000e-13,NaN,NaN,NaN,NaN,NaN
2,2022/11/08 10:58:17,41.0,4.880000e-13,2022/11/08 10:58:20,3.100000e-15,2022/11/08 10:58:22,9.100000e-15,2022/11/08 10:58:23,1.036000e-13,2022/11/08 10:58:24,7.310000e-14,2022/11/08 10:58:26,1.075000e-13,2022/11/08 10:58:28,4.966000e-13,NaN,NaN,NaN,NaN,NaN
3,2022/11/08 10:58:29,53.0,5.153000e-13,2022/11/08 10:58:32,2.630000e-14,2022/11/08 10:58:33,1.540000e-14,2022/11/08 10:58:35,8.420000e-14,2022/11/08 10:58:36,2.290000e-14,2022/11/08 10:58:38,4.700000e-15,2022/11/08 10:58:39,3.740000e-14,NaN,NaN,NaN,NaN,NaN
4,2022/11/08 10:58:42,66.0,4.988000e-13,2022/11/08 10:58:43,4.920000e-14,2022/11/08 10:58:45,4.600000e-15,2022/11/08 10:58:47,1.860000e-14,2022/11/08 10:58:48,3.010000e-14,2022/11/08 10:58:49,4.780000e-14,2022/11/08 10:58:50,8.200000e-15,NaN,NaN,NaN,NaN,NaN


In [3]:
#reading the Emission current, pressures
vaclog=pd.read_csv("pressure log_series2", sep="\t")
vaclog.head()
#adding an elapsed time column
timestamp=[]
for i in vaclog["Time"]:
    timestamp.append(i)
#making the column into a datetime object
timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
vaclog.drop(index=vaclog.index[0], 
        axis=0, 
        inplace=True)

vaclog.head()

,Live comments,Time,Elapsed time,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-platinum,T-CERNOX,I_emission,I_grid
1,NaN,08/11/2022 10:53:10,9.0,-0.033671,7.830000e-09,2.160000e-09,4.996000e-09,0.000002,0.001407,0.0,-113.355,54.770,0.0,NaN
2,NaN,08/11/2022 10:53:19,18.0,-0.033867,7.770000e-09,2.160000e-09,4.996000e-09,0.000002,0.001386,0.0,-113.243,54.545,0.0,NaN
3,NaN,08/11/2022 10:53:28,27.0,-0.034162,7.790000e-09,2.170000e-09,4.996000e-09,0.000002,0.001411,0.0,-113.243,54.657,0.0,NaN
4,NaN,08/11/2022 10:53:37,36.0,-0.033844,7.740000e-09,2.150000e-09,4.996000e-09,0.000002,0.001417,0.0,-113.355,54.545,0.0,NaN
5,NaN,08/11/2022 10:53:46,45.0,-0.033867,7.760000e-09,2.150000e-09,4.996000e-09,0.000002,0.001393,0.0,-113.467,54.545,0.0,NaN


In [4]:
#Reading in the HiVolta measurement file to get the grid current
hv = pd.read_csv("hv log_series2", sep=",")
#hv.drop(columns=["Vmon1","Vmon2","Vmon3","Vmon4","Vmon5","Vmon6","Vmon7","Vmon8","Imon3","Imon4","Imon5","Imon6","Imon7","Imon8","Comments1"], inplace=True)
hv["hv_grid"]=[element * 1e-6 for element in hv["IMon1"]]
hv["em"]=[element * 1e-6 for element in hv["IMon2"]]
hv.drop(hv.columns[hv.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)

#adding an elapsed time column
timestamp=[]
for i in hv["Time"]:
    timestamp.append(i)
    
#making the column into a datetime object
timestamps=pd.to_datetime(hv["Time"],format="%H:%M:%S.%f")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
hv.insert(2,"Elapsed time",runtime)
hv.drop(index=hv.index[0], 
        axis=0, 
        inplace=True)

hv.head()



,Date,Time,Elapsed time,VMon1,VMon2,VMon3,VMon4,VMon5,VMon6,VMon7,...,IMon2,IMon3,IMon4,IMon5,IMon6,IMon7,IMon8,Comment,hv_grid,em
1,08/11/2022,10:46:54.266,1.202,499.90,100.10,999.88,1000.00,599.9,0.0,0.0,...,-0.0049,35.9844,36.0013,35.9716,-0.0166,-0.0248,-0.0187,NaN,2.010000e-08,-4.900000e-09
2,08/11/2022,10:46:55.470,2.406,499.90,100.10,999.88,1000.00,599.9,0.0,0.0,...,-0.0049,35.9844,36.0013,35.9715,-0.0166,-0.0248,-0.0187,NaN,2.010000e-08,-4.900000e-09
3,08/11/2022,10:46:56.660,3.596,499.90,100.10,999.88,1000.00,599.9,0.0,0.0,...,-0.0049,35.9850,36.0013,35.9715,-0.0166,-0.0248,-0.0187,NaN,2.020000e-08,-4.900000e-09
4,08/11/2022,10:51:26.181,273.117,0.02,0.06,999.72,0.04,0.0,0.0,0.0,...,-0.0214,10.3697,-0.0283,-0.3344,-0.0170,-0.0232,-0.0174,NaN,2.600000e-08,-2.140000e-08
5,08/11/2022,10:51:27.367,274.303,0.02,0.06,999.72,0.04,0.0,0.0,0.0,...,-0.0214,10.3694,-0.0286,-0.3371,-0.0170,-0.0232,-0.0174,NaN,2.600000e-08,-2.140000e-08


### Data processing

In [32]:
#Filtering background signal

#Find the timestamp where emission was turned on
print("Result 1:",hv.loc[hv["Comment"]=="emission start"])
x=pd.Timestamp(2022, 11, 8, 11,3, 46)

#then find the time value closest to it in the MID scan file
print("\n","Result 2:",mid.iloc[(pd.to_datetime(mid['mass 2_time'],format="%Y/%m/%d %H:%M:%S") - x).abs().argsort(),:])

#average this signal and subtract it from the Em-on part
bck=np.mean(mid["mass 2_value"][0:28])
print("\n","Result 3:",bck)

#apply background correction to the rest of the mass spectrum
mid["mass 2, corr"]=mid["mass 2_value"][29:139]-bck
mid["mass 2, corr"].describe()

Result 1:            Date          Time  Elapsed time  VMon1   VMon2   VMon3   VMon4  \
626  08/11/2022  11:03:46.639      1013.575   0.10  100.08  999.84  999.72   
627  08/11/2022  11:03:47.831      1014.767   0.10  100.08  999.84  999.72   
628  08/11/2022  11:03:49.038      1015.974   0.08  100.10  999.82  999.72   
629  08/11/2022  11:03:50.232      1017.168   0.08  100.08  999.86  999.72   

      VMon5  VMon6  VMon7  ...   IMon2    IMon3    IMon4    IMon5   IMon6  \
626  999.68    0.0    0.0  ...  1.1505  37.8549  27.4654  27.2002 -0.0176   
627  999.68    0.0    0.0  ...  1.2188  37.8467  27.4604  27.1347 -0.0176   
628  999.66    0.0    0.0  ...  1.0954  37.9896  27.6060  27.3130 -0.0176   
629  999.68    0.0    0.0  ...  1.2876  37.7281  27.3448  27.0351 -0.0176   

      IMon7   IMon8         Comment       hv_grid        em  
626 -0.0238 -0.0182  emission start -1.258000e-07  0.000001  
627 -0.0238 -0.0182  emission start -1.184000e-07  0.000001  
628 -0.0238 -0.0182  emissi

count    1.100000e+02
mean     5.943703e-13
std      1.739281e-13
min     -9.231429e-14
25%      5.082357e-13
50%      6.201857e-13
75%      6.783357e-13
max      1.003486e-12
Name: mass 2, corr, dtype: float64

In [37]:
#getting the differential spectrum
#first get the timestamp where grid voltage was changed to -500V
print("Location:",hv.loc[hv["Comment"]=="grid repelling"])
y=pd.Timestamp(2022, 11, 8, 11,17, 47)

#then find the time value closest to it in the MID scan file
print("\n","Result 2:",mid.iloc[(pd.to_datetime(mid['mass 2_time'],format="%Y/%m/%d %H:%M:%S") - y).abs().argsort(),:])

#define the differential spectrum
#how to implement when slices are different size?
#does not work !!! mid["mass 2, diff"]=mid["mass 2, corr"][29:96]-mid["mass 2, corr"][97:140]
#mid["mass 2, diff"]

Location:             Date          Time  Elapsed time   VMon1  VMon2   VMon3    VMon4  \
1330  08/11/2022  11:17:47.305      1854.241  421.34  100.1  999.92  1000.02   
1331  08/11/2022  11:17:48.514      1855.450  499.20  100.1  999.92  1000.02   
1332  08/11/2022  11:17:49.745      1856.681  499.40  100.1  999.92  1000.02   
1333  08/11/2022  11:17:50.942      1857.878  499.60  100.1  999.92  1000.02   

        VMon5  VMon6  VMon7  ...   IMon2    IMon3    IMon4    IMon5   IMon6  \
1330  1000.04    0.0    0.0  ...  0.0432  40.3153  29.9536  29.6362 -0.0146   
1331  1000.04    0.0    0.0  ...  0.0363  40.3080  29.9507  29.6358 -0.0146   
1332  1000.04    0.0    0.0  ...  0.0284  40.3051  29.9516  29.6368 -0.0146   
1333  1000.04    0.0    0.0  ...  0.0253  40.3039  29.9536  29.6392 -0.0146   

       IMon7   IMon8         Comment       hv_grid            em  
1330 -0.0227 -0.0174  grid repelling  5.632000e-07  4.320000e-08  
1331 -0.0227 -0.0174  grid repelling  1.819000e-07  3.63000

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
       ..
135   NaN
136   NaN
137   NaN
138   NaN
139   NaN
Name: mass 2, diff, Length: 140, dtype: float64

### Plotting

In [14]:
#Plotting the all the spectrums overlaid on top
plt.figure(figsize=(14,8))
plt.plot(mid['Elapsed time, mass 2'],mid["mass 2_value"],marker=".", markersize=4,label='h2')


plt.xlabel('Elapsed time (s)')
plt.ylabel('Ion current (A)')
plt.legend(title="Mol masses", loc="upper right")
plt.title("RGA scans overlay")
plt.yscale('log')


In [ ]:
#Plotting Emission on, grid transp. - Emission on, grid repelling
#essentially total ESD= EGA ESD + sample ESD
#sample ESD= Emission on, grid transp - bckground,grid repelling
plt.figure(figsize=(12,6))
plt.plot((dif["Mass"]),dif["sig"],marker=".", markersize=4,label='differential spectrum')
plt.xlabel('Mass fraction (u)')
plt.ylabel('Ion current (A)')
plt.legend(loc="upper right")
plt.title("Differential mass spectrum of ESD from the copper sample")
plt.xticks(np.arange(0,52,2.0),rotation='vertical')
plt.yscale('log')
plt.ylim(1e-15,2e-13)

In [ ]:
#plotting HiVolta log
plt.figure(figsize=(12,6))
plt.plot(hv["Elapsed time"]/3600,hv["IMon1"],marker=".", markersize=4,label='Grid current')
plt.plot(hv["Elapsed time"]/3600,hv["IMon2"],marker=".", markersize=4,label='Emission current')
plt.xlabel('Elapsed time (h)')
plt.ylabel('Current (A)')
plt.legend(loc="upper right")
plt.title("HiVolta current recording")
plt.yscale('log')


In [ ]:
#Plotting BA2, BA1 pressure
plt.figure(figsize=(10,6))
plt.plot(vaclog["Elapsed time"]/3600,vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2 pressure reading')
plt.plot(vaclog["Elapsed time"]/3600,vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1 pressure reading')

plt.xlabel('Elapsed time (h)')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("Barion 2 pressure evolution")
ax = plt.gca()
ax.set_yscale('log')
plt.tick_params(axis="y", which='minor')
ax.yaxis.set_minor_formatter(LogFormatter(minor_thresholds=(2,0.5)))
plt.show()

# Primary desorption calculation method
### Elena's formula for the cryo setup
$$
\eta=\frac{C_{H2}\cdot (\Delta p_1 - \Delta p_2)q_e}{k_B\cdot T \cdot I_e}
$$

Where $\Delta p_i=p(t)_i - p(t=0)_i$ and $i=1,2$
In the case of the 4K setup, we know the exact conductance of gasses. The formula above applies well.


In [ ]:
#calculation delta_p-s
dp1 = np.mean(vaclog["Barion_1"].nlargest(20))-vaclog["Barion_1"].min()*100
dp2 = np.mean(vaclog["Barion_2"].nlargest(20))-vaclog["Barion_2"].min()*100
print(dp1, dp2)

#Constants
k_B = 1.38e-23
T = 296
q_e = 1.6e-19
c_h2 = 37.91/1000
#Emission current
I_em = np.mean(hv["Imon2"].nlargest(30))*1e-6
print(I_em)
#Conductance for CO2 in m3/s
c_co2 = 8.08/1000


In [ ]:
#Desorption yield
eta_h2 = (c_h2*(dp1-dp2)*q_e)/(k_B*T*I_em)
eta_co2 = (c_co2*(dp1-dp2)*q_e)/(k_B*T*I_em)
print("The desorption yield for H2 [mol/el]:",eta_h2,"\n","The desorption yield for CO2 [mol/el]:",eta_co2)